In [1]:
import numpy as np
import seaborn as sns
import pandas as pd
import seaborn.objects as so
import matplotlib.pyplot as plt

from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score    # Medidas de desempeño
from sklearn.preprocessing import PolynomialFeatures    # Herramientas de polinomios

# Esto es para que no me aparezcan los warnings cuando ejecuto los modelos
import warnings
warnings.filterwarnings("ignore")

## Ejercicio 1

#### Queremos estudiar la relacion entre la longitud de la aleta de un pinguino y el peso del pinguino. Como en una esfera, el peso es proporcional a la longitud del radio elevada al cubo, podemos conjeturar que un polinomio de grado 3 es apropiado para ajustar el peso en funci on de la longitud de la aleta. Queremos verificar si nuestra conjetura tiene sustento en los datos

### A) Datos faltantes. Eliminar las filas con datos faltantes del DataFrame de ping ̈uinos y verificar que el DataFrame resultante no contiene valores faltantes

In [5]:
penguins = sns.load_dataset("penguins")
penguins.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female


Se puede ver que hay. Sino ejecutamos 
``` bash
penguins.isnull ().values.any()
```

In [8]:
print(f"Antes de eliminar los valores nulos: {penguins.shape}")
penguins.dropna(inplace=True)
print(f"Después de eliminar los valores nulos: {penguins.shape}")
print(f"Verifico si hay valores nulos: \n{penguins.isnull().sum()}")

Antes de eliminar los valores nulos: (333, 7)
Después de eliminar los valores nulos: (333, 7)
Verifico si hay valores nulos: 
species              0
island               0
bill_length_mm       0
bill_depth_mm        0
flipper_length_mm    0
body_mass_g          0
sex                  0
dtype: int64


### (B) Conjuntos de entrenamiento y testeo. 
Dividir el dataset resultante en un grupo de
entrenamiento y uno de testeo (80% - 20%). Hacerlo de las siguientes dos formas distintas:

A. Utilizando un array de Numpy para filtrar:

i. Crear un array de Numpy booleano de longitud igual a la cantidad de filas del DataFrame que tome el valor True en el primer 80% de las coordenadas y `False` en el restante 20%.

ii. Si se quiere seleccionar una muestra al azar, se puede utilizar `numpy.random.shuffle` para “mezclar” el vector.
    
iii. Utilizar el vector generado para filtrar el Dataframe.

In [15]:
# i)
train_ind = np.full(penguins.shape[0], False)
train_ind[0:int(len(train_ind) * 0.8)] = True
# ii)
np.random.shuffle(train_ind) # Lo guarda en el mismo vector.
# iii)
penguins_train = penguins[train_ind]
penguins_test = penguins[~train_ind]

B. Utilizando la funcion `train_test_split` de sklearn:

In [16]:
from sklearn.model_selection import train_test_split
penguins_train, penguins_test = train_test_split(penguins, test_size=0.2, random_state=42)

Tambien se puede hacer asi:


In [27]:
X = penguins["flipper_length_mm"] # Columna/s predictoras. En este caso: la longitud de las aletas
y = penguins["body_mass_g"]# Columna/s objetivo. En este caso: el peso del pingüino
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### (c) Crear y ajustar 3 modelos utilizando polinomios de grados 1, 2 y 3.

In [28]:
X_train = X_train.values.reshape(-1, 1)

modelo_1 = linear_model.LinearRegression()
modelo_1.fit(X_train, y_train)

modelo_2 = linear_model.LinearRegression()
polynomial_features2= PolynomialFeatures(degree=2, include_bias=False) 
x_poly = polynomial_features2.fit_transform(X_train)
modelo_2.fit(x_poly, y_train)

modelo_3 = linear_model.LinearRegression()
polynomial_features3= PolynomialFeatures(degree=3, include_bias=False)
x_poly = polynomial_features3.fit_transform(X_train)
modelo_3.fit(x_poly, y_train)


LinearRegression()

### (d) Calcular para cada uno el error prediccion en el grupo de entrenamiento y en el grupo de test

In [29]:
X_test = X_test.values.reshape(-1, 1)

In [32]:
print(f"ECM modelo 1, en entrenamiento: {mean_squared_error(y_train, modelo_1.predict(X_train))}")
print(f"ECM modelo 1, en prueba: {mean_squared_error(y_test, modelo_1.predict(X_test))}\n")

print(f"ECM modelo 2, en entrenamiento: {mean_squared_error(y_train, modelo_2.predict(polynomial_features2.fit_transform(X_train)))}")
print(f"ECM modelo 2, en prueba: {mean_squared_error(y_test, modelo_2.predict(polynomial_features2.fit_transform(X_test)))}\n")

print(f"ECM modelo 3, en entrenamiento: {mean_squared_error(y_train, modelo_3.predict(polynomial_features3.fit_transform(X_train)))}")
print(f"ECM modelo 3, en prueba: {mean_squared_error(y_test, modelo_3.predict(polynomial_features3.fit_transform(X_test)))}\n")

ECM modelo 1, en entrenamiento: 159873.09549908544
ECM modelo 1, en prueba: 129886.71972544755

ECM modelo 2, en entrenamiento: 147737.6311072205
ECM modelo 2, en prueba: 127731.22567375602

ECM modelo 3, en entrenamiento: 143995.99465654948
ECM modelo 3, en prueba: 123664.64922778215

